In [1]:
!pip install folium
!pip install geopandas

In [2]:
# 今回はこのデータを使っている https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-P32.html
import geopandas as gpd
cultural_gpd = gpd.read_file('../data/P32-14_15_GML')

dic_type = {
    '1': '有形文化財',
    '2': '無形文化財',
    '3': '民俗文化財',
    '4': '記念物',
    '5': '文化的景観',
    '6': '伝統的建造物群',
    '7': '文化財の保存技術'
}
dic_color = {
    '1': 'black',
    '2': 'orange',
    '3': 'blue',
    '4': 'purple',
    '5': 'red',
    '6': 'pink',
    '7': 'green'
}

cultural_gpd['種別大分'] = cultural_gpd['P32_004'].astype(str).map(dic_type)
cultural_gpd.head()

,P32_001,P32_002,P32_003,P32_004,P32_005,P32_006,P32_007,P32_008,P32_009,geometry,種別大分
0,101,15,15205,1,11,飯縄社本殿,柏崎市大清水,19521210,3,POINT (138.42824 37.30172),有形文化財
1,201,15,15227,1,11,弁天堂,胎内市乙,19550209,3,POINT (139.40443 38.11679),有形文化財
2,301,15,15206,1,11,市島家住宅,新発田市天王,19620329,3,POINT (139.27298 37.89959),有形文化財
3,401,15,15212,1,11,西奈弥神社境内摂社神明宮本殿,村上市大字村上,19690325,3,POINT (139.48306 38.23505),有形文化財
4,501,15,15224,1,11,小比叡神社拝殿,佐渡市小比叡,19730329,3,POINT (138.27718 37.83530),有形文化財


In [3]:
cultural_gpd['種別大分'].value_counts()

有形文化財       222
記念物         108
民俗文化財        37
文化財の保存技術     17
無形文化財         7
Name: 種別大分, dtype: int64

In [4]:
def make_popup_html(row):
    popup=folium.Popup(html = f"""
        <table border="1">
            <caption> {row['P32_006']}<br></caption>
            <tr>
                <td>所在地
                <td>{row['P32_007']}</td>
            </tr>
            <tr>
                <td>種別大分</td>
                <td>{row['種別大分']}</td>
            </tr>
        </table>
    """,
    max_width=400,
    )
    return popup

In [6]:
import folium
# 対象駅のデータ
from folium import FeatureGroup, LayerControl

m = folium.Map(location=['37.5178', '138.9270'], zoom_start=10)

#　商圏エリアと候補駅
for index_i, prop_type in enumerate(cultural_gpd['種別大分'].value_counts().index):
    show = False
    if prop_type in ['有形文化財', '無形文化財']:
        show=True
    temp_group = FeatureGroup(name=prop_type, show=show) # レイヤコントロールで操作するグループを作成する
    for index_j, row in cultural_gpd[cultural_gpd['種別大分']==prop_type].iterrows():

        folium.Marker(
        location=[row['geometry'].y, row['geometry'].x],
        popup=make_popup_html(row),
        icon=folium.Icon(color=dic_color[str(row['P32_004'])]),
        ).add_to(temp_group)
        
    temp_group.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
m